In [1]:
from PySide.QtGui import QApplication
from viewer.vtkviewer import SimpleVtkViewer

import numpy as np

from occmodel import Edge, EdgeIterator, Wire, Face, FaceIterator, Solid
from geotools import Plane

### [OCCT Bottle Tutorial](http://dev.opencascade.org/doc/overview/html/occt__tutorial.html)

In [2]:
height = 70.
width = 50.
thickness = 30.

pnt1 = [-width/2.,  0.,           0.]
pnt2 = [-width/2., -thickness/4., 0.]
pnt3 = [0.,        -thickness/2., 0.]
pnt4 = [width/2.,  -thickness/4., 0.]
pnt5 = [width/2.,   0.,           0.]

edge1 = Edge().createLine(start=pnt1, end=pnt2)
edge2 = Edge().createArc3P(start=pnt2, end=pnt4, pnt=pnt3)
edge3 = Edge().createLine(start=pnt4, end=pnt5)

halfProfile = Wire([edge1, edge2, edge3])

mirrorPlane = Plane(origin=[0,0,0], xaxis=[1,0,0], yaxis=[0,0,1])
mirrorProfile = halfProfile.mirror(mirrorPlane, copy=True)

allEdges = list(EdgeIterator(halfProfile)) + list(EdgeIterator(mirrorProfile))

fullProfile = Wire().createWire(allEdges)

bottomFace = Face().createFace(fullProfile)

body = Solid().extrude(bottomFace, (0, 0, 0), (0, 0, height))
body.fillet(thickness/12.)

neckHeight = height/10
neckRadius = thickness/4
neck = Solid().createCylinder([0,0,0], [0,0,neckHeight], radius=neckRadius)
neck.translate([0, 0, height])

body.fuse(neck)

zMax = -1
neckTopFace = None
for f in FaceIterator(body):
    [x, y , z] = f.centreOfMass()
    if z >= zMax:
        neckTopFace = f
        zMax = z
        
body.shell(thickness/50., [neckTopFace], tolerance=1E-3)

t_thick = neckHeight/5
t_height = neckHeight - t_thick
t_radius = neckRadius + t_thick/4
t_pitch = t_height/2
t_angle = 0

# Note the following thread geometry is not correct.  The profile
# is wrong and there is a twist added to the profile. But it's
# kind of close and good enough for this example.
threadHelix = Edge().createHelix(pitch=t_pitch, 
                                 height=t_height,
                                 radius=t_radius,
                                 angle = t_angle)

threadFace = Face().createPolygonal([[0,    0,        t_thick/2], 
                                     [t_thick,  .0,   0], 
                                     [0,    0,      -t_thick/2]])
threadFace.translate([t_radius, 0, 0])

thread = Solid().pipe(threadFace, threadHelix)
thread.translate([0, 0, height])

body.fuse(thread)

actor = body.toVtkActor()

In [3]:
%gui qt

try:
    a = QApplication([])
except:
    pass

vtkWin = SimpleVtkViewer()

vtkWin.add_actor(actor)

# If the VTK window is blank/white, click on the window and hit 'r' 
# to zoom to fit.